# Imports

In [4]:
import pandas as pd

from src import CTX
from src.features import COLUMNS, build_features
from src.data import io, FOLDERS

In [5]:
observations = io.load_data(
    FOLDERS.PROCESSED,
    CTX + 'observations',
    pd.DataFrame())
observations

Loading ../data/processed/CC-Labs-hv_observations.h5


,Observation
0,"Tens of categories with very few items, a hand..."
1,First word of category name a good candidate f...
2,(Categories already sorted on category name)
3,First word of shop name a good candidate for s...
4,(Shops already sorted on shop name)
5,"All shop, item, and category references are va..."
6,Training data does not have explicit NULLs
7,Submissions are evaluated by root mean squared...
8,For each ID in the test set (shop_id/item_id c...
9,One in seven test data keys do not have entrie...
